In [2]:
# Seeing if PySpark works

import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.14540416


In [ ]:
import json
import sys, os
import argparse
import numpy as np
import findspark
findspark.init()
import pyspark
import random
import pandas as pd

#from multiprocessing import Pool

# Getting config
with open('/dfs/scratch1/jdunnmon/data/memex-data/config/config_spark.json') as fl:
    config = json.load(fl)

# Adding path for utils
sys.path.append('/dfs/scratch1/jdunnmon/repos/extractors/src/utils')

# Setting random seed
seed = config['seed']
random.seed(seed)
np.random.seed(seed)

# If memex_raw_content is a content_field, uses term as a regex in raw data in addition to getting title and body
#term = r'\b[Ll]ocation:|\b[cC]ity:'
#term = r'([Ll]ocation:.{0,100}|[cC]ity:.{0,100}|\d\dyo\W|\d\d.{0,10}\Wyo\W|\d\d.{0,10}\Wold\W|\d\d.{0,10}\Wyoung\W|\Wage\W.{0,10}\d\d)'
term = r'([Ll]ocation:[\w\W]{1,200}</.{0,20}>|\W[cC]ity:[\w\W]{1,200}</.{0,20}>|\d\dyo\W|\d\d.{0,10}\Wyo\W|\d\d.{0,10}\Wold\W|\d\d.{0,10}\Wyoung\W|\Wage\W.{0,10}\d\d)'

# Setting up arguments (can also get from argparse)
args = {}
args['data_loc'] = '/dfs/scratch1/jdunnmon/data/memex-data/tsvs/output_all_slicetest'
# Getting raw_content column
print('Getting correct column...')
files = os.listdir(args['data_loc'])
df = pd.read_csv(os.path.join(args['data_loc'],files[0]),sep='\t',nrows=10)
col = df.columns.get_loc("memex_raw_content")

# Getting path and setting up data
path = args['data_loc']
file_list = os.listdir(path)
path_list = [os.path.join(path, file) for file in file_list]
file_data = [(path, term, col) for path in path_list if path.endswith('tsv')]

out_dir = path + '/parsed/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

print('Processing in parallel')


sc = pyspark.SparkContext(appName="parse_html")
sc.addPyFile('/dfs/scratch1/jdunnmon/repos/extractors/src/utils/dataset_utils.py')
from dataset_utils import parse_html
print('Distributing data...')
distData = sc.parallelize(file_data)
print('Parsing distributed data...')
distData.foreach(parse_html)
sc.stop()
#threads = 30
#pool = Pool(threads) 
#results = pool.map(parse_html, file_data)
#pool.close()
#pool.join()


Getting correct column...
Processing in parallel
Distributing data...
Parsing distributed data...


In [5]:
sc.stop()

In [3]:
import json
import gzip
pth='/lfs/local/0/jdunnmon/data/memex-data/escorts/2015/05/08/0000.jsonl.gz'
with gzip.GzipFile(pth, 'r') as in_file:
        
        # Getting file objects
        json_reader = in_file.read()
        
        for ii, chunk in enumerate(json_reader.splitlines()):
            data = json.loads(chunk)
            break

In [6]:
data.keys()

dict_keys(['type', 'crawl_data', 'url', 'timestamp', 'extractions', 'raw_content', 'extracted_metadata', 'version', 'extracted_text', 'content_type', 'team', 'doc_id', 'crawler'])

In [7]:
field_dict = {'id':'doc_id', 'uuid':'', 'memex_id':'doc_id', 'memex_doc_type':'type', 'memex_raw_content':'raw_content', 'memex_url':'url', 'url':'', 'extractions':'extractions'}

In [8]:
field_dict.keys()

dict_keys(['id', 'uuid', 'memex_id', 'memex_doc_type', 'memex_raw_content', 'memex_url', 'url', 'extractions'])

In [1]:
import sys, os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
sys.path.append('../utils')
from dataset_utils import parse_html_from_jsonl_gz

in_loc = '/lfs/local/0/jdunnmon/data/memex-data/escorts/2015/05/08/0000.jsonl.gz'
in_parts = in_loc.split('/')
in_parts[in_parts.index('escorts')] = 'escorts_preproc'
in_parts[-1] = in_parts[-1].split('.')[0]+'.tsv'
out_loc = '/'.join(in_parts)
term = r'([Ll]ocation:[\w\W]{1,200}</.{0,20}>|\W[cC]ity:[\w\W]{1,200}</.{0,20}>|\d\dyo\W|\d\d.{0,10}\Wyo\W|\d\d.{0,10}\Wold\W|\d\d.{0,10}\Wyoung\W|\Wage\W.{0,10}\d\d)'
field_dict = {'id':'doc_id', 'uuid':'', 'memex_id':'doc_id', 'memex_doc_type':'type', 'memex_raw_content':'raw_content', 'memex_url':'url', 'url':'', 'extractions':'extractions'}
content_field = 'memex_raw_content'
file_data = (in_loc, out_loc, term, field_dict, content_field)
parse_html_from_jsonl_gz(file_data)

Error on line: 293
Error on line: 2759
Error on line: 4247
Error on line: 6240
Error on line: 8249
Error on line: 8801
Error on line: 10022
Error on line: 10517
Error on line: 10518
Error on line: 10954
Error on line: 11458
Error on line: 11705
Error on line: 13726
Error on line: 14000
Error on line: 14444
Error on line: 15012
Error on line: 15311
Error on line: 17569
Error on line: 18056
Error on line: 18318
Error on line: 18325
Error on line: 18366
Error on line: 18961
Error on line: 20161
Error on line: 20485
Error on line: 20843
Error on line: 21346
Error on line: 21535
Error on line: 22567
Error on line: 22639
Error on line: 22641
Error on line: 24061
Error on line: 25125
Error on line: 25774
Error on line: 25798
Error on line: 29836
Error on line: 31447
Error on line: 31460
Error on line: 31594
Error on line: 31840
Error on line: 32446
Error on line: 32659
Error on line: 35157
Error on line: 36897
Error on line: 38125
Error on line: 38569
Error on line: 39943
Error on line: 40149

In [ ]:
lin